# Śledzenie obiektów

<img src="https://i.imgur.com/wKXXFkQ.png" width="500">

## Wstęp
W erze cyfrowej, w obliczu rosnącej lawinowo ilości danych wideo, zdolność do ich automatycznego rozpoznawania i interpretowania staje się kluczowa w wielu dziedzinach – od bezpieczeństwa publicznego po autonomiczne pojazdy. Technologie oparte na głębokim uczeniu rewolucjonizują sposób, w jaki przetwarzamy informacje wizualne. Kluczowym wyzwaniem jest tu detekcja i śledzenie obiektów na filmach wideo.

Celem tego zadania jest opracowanie algorytmu, który będzie w stanie analizować sekwencje ruchów w grze "trzy kubki". Uczestnicy mają za zadanie określić końcową pozycję kubków po serii ruchów, korzystając z analizy statycznych obrazów z każdej klatki nagrania.

In [2]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Poniższe funkcje ułatwiają pracę z dostarczonymi danymi
# W kolejnych komórkach zobaczysz przykłady ich użycia
from utils.utils import get_level_info, get_video_data, display_video, download_and_replace_data

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!
# if not FINAL_EVALUATION_MODE:
#     images, coordinates, target, path_to_images = get_video_data(level=2,video_id=0,dataset="example")
#     display_video(images,rescale=0.7,FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

## Zadanie 2: Okluzje, rozmycia, przesłonienie

Celem zadania jest opracowanie algorytmu, który potrafi przetwarzać sekwencje obrazów z gry w trzy kubki, nawet gdy występują rozmycia czy przesłonięcia. Zadanie ma na celu nauczenie maszyny wykorzystywania ciągłości informacji z kolejnych klatek, aby mimo chwilowych utrudnień w percepcji, mogła skutecznie określić końcową pozycję kubków.

Napisz algorytm który poradzi sobie z trudnieszym zbiorem danych - `Level_2`. Składa się on z animacji, w których pojawiają się dodatkowe utrudnienia:
- Przesłonięcia obiektu przez inny, powodujące, że są one traktowane jako jeden,
- Prostokąty ograniczające nie są już idealnie dopasowane do obiektów,
- Prostokąty ograniczające nie są widoczne we wszystkich klatkach.

Będziesz miał dostęp zarówno do wszystkich klatek animacji, jak i do oznaczonych przez nas prostokątów ograniczających, w których znajdują się kubki. Co ważne, algorytm, który będziesz tworzył ma korzystać jedynie z informacji o prostokątach ograniczających. W tym zadaniu, klatki wideo są dostarczone jedynie do wizualizacji przykładów i algorytmu, na własne potrzeby.

Punkty za to zadanie będą przyznane za osiągnięcie jak najdokładniejszych predykcji na zbiorze testowym. Kryterium będzie *accuracy* i spodziewamy się wyników powyżej `80%`. Ewaluacja na zbiorze testowym będzie dokonana przez organizatorów.

## Pliki zgłoszeniowe
Tylko ten notebook zawierający **kod** oraz **krótki raport** opisujący Twoje rozwiązanie (do 300 słów). Miejsce na raport znajdziesz na końcu tego notebooka.

## Ograniczenia
- Twoja funkcja powinna zwracać predykcje w maksymalnie 5 minut używając Google Colab bez GPU.

## Uwagi i wskazówki
- Testuj swoje rozwiązanie na zbiorze plików wideo `level_2`.
- **Skuteczność modelu**: przetestuj skuteczność modelu na zbiorze walidacyjnym używając dostarczonej przez nas funkcji **submission_script**, umieść ten wynik w raporcie.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to podzadanie możesz zdobyć pomiędzy 0 i 0.5 punktów. Zdobędziesz 0 punktów jeśli Twoje accuracy na zbiorze testowym będzie poniżej 50%. Jeśli będzie większe niż 95%, otrzymasz 0.5 punktu. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością metryki.

# Kod startowy

In [3]:
# Poniższe biblioteki są wystarczające do wykonania wszystkich zadań
# Jeśli jednak chcesz użyć innych, sprawdź czy są dostępne na serwerze (requirements.txt)
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import IPython.display
import json
import PIL
import sklearn as sk

In [4]:
# Pobieranie danych do podzadań 1, 2 i 3 (około ~646Mb), skrypt będzie wykonywał się parę minut
# Wystarczy, że pobierzesz dane tylko raz. Na serwerze sprawdzającym dane będą już pobrane,
# struktura plików będzie identyczna jak tutaj
if not FINAL_EVALUATION_MODE:
    download_and_replace_data()

Downloading...
From (original): https://drive.google.com/uc?id=1gEV52fT3luVkTU_Qf-aTd2hWj-7rYCpQ
From (redirected): https://drive.google.com/uc?id=1gEV52fT3luVkTU_Qf-aTd2hWj-7rYCpQ&confirm=t&uuid=34b4e803-2177-47de-ae22-37e58b2ddd56
To: /content/train_data.zip
100%|██████████| 444M/444M [00:05<00:00, 80.9MB/s]


Extracted and cleaned up train_data.zip


Downloading...
From (original): https://drive.google.com/uc?id=1JDI4nWtIYlBp56QPbS3lsGyxZA9vHw5t
From (redirected): https://drive.google.com/uc?id=1JDI4nWtIYlBp56QPbS3lsGyxZA9vHw5t&confirm=t&uuid=d69d6a6e-b595-4831-9841-2c5b3f60ed0b
To: /content/valid_data.zip
100%|██████████| 233M/233M [00:02<00:00, 105MB/s]


Extracted and cleaned up valid_data.zip


In [5]:
# funkcja pomocnicza do ładowania danych
if not FINAL_EVALUATION_MODE:
    images, _, _, _ = get_video_data(level=2,video_id=1,dataset="train")

    # with open(os.path.join(os.getcwd(),'example_tracks','tracks_2_0.json'), 'r') as f:
    #     tracks = json.load(f)

    # for key in tracks.keys():
    #     tracks[key] = [tuple(el) for el in tracks[key]]

    # funkcja pomocnicza do wyświetlania danych
    display_video(images,
                    rescale=0.7,
                    FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)


In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# funkcja pomocnicza do testowania algorytmu
def submission_script(algorithm,level,verbose=False,dataset="valid"):
    num_videos, _ = get_level_info(level=level,dataset=dataset)
    correct = []
    exception_messages = set()
    for video_number in range(num_videos):
        _, coordinates, target, _ = get_video_data(level=level,video_id=video_number,dataset=dataset)
        try:
            prediction = algorithm(coordinates)
            if tuple(target) == tuple(prediction):
                correct.append(1)
            else:
                correct.append(0)
            if verbose:
                print(f"Video: animation_{str(video_number).zfill(4)}")
                print(f"Prediction: {prediction}")
                print(f"Target:     {target}")
                print(f"Score: {tuple(target) == tuple(prediction)}", end='\n\n')
        except Exception as e:
            correct.append(0)
            exception_messages.add(str(e))
    if verbose:
        print(f"Accuracy: {np.mean(correct)}")
        print(f"Correctness: {correct}")
    return np.sum(correct) / num_videos, correct, exception_messages

# Twoje rozwiązanie

In [7]:
def your_algorithm_task_2(coordinates): # nie zmieniaj nazwy funkcji
    def get_order(frame):
        frame = np.array(frame)

        if frame[0,0] > frame[1,0] and frame[1,0] > frame[2,0]:
            return [2,1,0]
        elif frame[1,0] > frame[0,0] and frame[0,0] > frame[2,0]:
            return [1,2,0]
        elif frame[0,0] > frame[2,0] and frame[2,0] > frame[1,0]:
            return [2,0,1]
        elif frame[1,0] > frame[2,0] and frame[2,0] > frame[0,0]:
            return [0,2,1]
        elif frame[2,0] > frame[0,0] and frame[0,0] > frame[1,0]:
            return [1,0,2]
        elif frame[2,0] > frame[1,0] and frame[1,0] > frame[0,0]:
            return [0,1,2]

    def distance(obj1, obj2):
        a = (((obj1[0]-obj2[0])**2) + ((obj1[1]-obj2[1])**2))**0.5

        return a

    def find_missing(table):
        missing = []
        if 0 not in table:
            missing.append(0)
        if 1 not in table:
            missing.append(1)
        if 2 not in table:
            missing.append(2)

        return missing

    def is_good_distance(arr):
        arr = sorted(arr)

        if(arr[0] == np.inf):
            return False
        if(arr[1] == np.inf):
            return True

        if arr[1] - arr[0] > 12.75:
            return True
        return False

    def get_distance(arr):
        arr = sorted(arr)

        if(arr[0] == np.inf):
            return -1
        if(arr[1] == np.inf):
            return np.inf

        return arr[1] - arr[0]

    def move(frame, prev_frame, order):

        if len(frame) == 0:
            return np.full(3,-1)

        output = np.full(3,-1)
        distances_matrix = np.full((3,3), np.inf)

        for i in range(len(frame)):
            for j in range(len(prev_frame)):
                distances_matrix[i,j] = distance(frame[i], prev_frame[j])

        distances_in_row = np.array([get_distance(distances_matrix[i]) for i in range(len(distances_matrix))])
        while True:
            row = np.argmax(distances_in_row)
            distances_in_row[row] = -1

            if row == -1:
                break

            flat_min = np.argmin(distances_matrix[row])
            idx = row,flat_min

            if(distances_matrix[idx] == np.inf):
                break


            if(is_good_distance(distances_matrix[idx[0]])):
                output[idx[0]] = order[idx[1]]
                distances_matrix[:,idx[1]] = np.inf

            distances_matrix[idx[0],:] = np.inf

        return output

    def calc_vector(box, prev_box, v2):
        v1 = np.array([box[0] - prev_box[0], box[1] - prev_box[1]])

        sr = (v1 + np.array(v2))/2

        v1 = resize_vector(v1, vector_len(sr))

        if(vector_len(v1) > 50):
            v1 = np.array([0,0])

        return v1

    def add_vector(center, vector):
        return [center[0] + vector[0], center[1] + vector[1]]

    def get_centers(arr):
        new_arr = []

        for el in arr:
            new_arr.append([(el[0] + el[2])/2, (el[1] + el[3])/2])

        return new_arr

    def vector_len(v1):
        return np.array((v1[0]**2 + (v1[1]**2))**0.5)

    def resize_vector(v1, n):
        if n==0:
            return v1/2
        if vector_len(v1) == 0:
            return np.array([0,0])

        epsilon = vector_len(v1)/n
        v1 = v1/epsilon
        return v1

    starting_order = get_order(coordinates["frame_0000.png"])
    current_order = starting_order
    prev_frame = get_centers(coordinates["frame_0000.png"])
    vectors = np.array([[0,0],[0,0],[0,0]], dtype=np.double)

    for frame in coordinates.keys():

        centers = get_centers(coordinates[frame])

        prev_order = current_order

        current_order = move(centers, prev_frame, current_order)

        new_frame = []

        missing = find_missing(current_order)
        missing_id = 0

        for i,el in enumerate(current_order):

            if el != -1:
                new_frame.append(centers[i])

                vectors[el] = calc_vector(new_frame[i], prev_frame[np.where(prev_order == el)[0][0]], vectors[el])
            else:
                miss_id = np.where(prev_order == missing[missing_id])[0][0]
                missing_id += 1

                current_order[i] = prev_order[miss_id]
                new_frame.append(add_vector(prev_frame[miss_id], vectors[prev_order[miss_id]]))

        prev_frame = np.array(new_frame)

    final_order = get_order(prev_frame)

    end = np.zeros(3)
    for i in range(len(final_order)):
        end[final_order[i]] = current_order[i]

    return list(end.astype(np.int16))

In [ ]:
# imgs, coordinates, _, _ = get_video_data(level=2, video_id=55, dataset="train")

# wynik, kp = your_algorithm_task_2(coordinates)

# display_video(imgs,
#               first_preds=kp[0],
#               second_preds=kp[1],
#               third_preds=kp[2],
#                 rescale=0.7,
#                 FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [8]:
# Sprawdź jak działa Twój algorytm
if not FINAL_EVALUATION_MODE:
    accuracy, correctness, _ = submission_script(
        algorithm=your_algorithm_task_2,
        level=2,
        verbose=True,
        dataset="valid")

Video: animation_0000
Prediction: [2, 0, 1]
Target:     [2, 0, 1]
Score: True

Video: animation_0001
Prediction: [0, 1, 2]
Target:     [0, 1, 2]
Score: True

Video: animation_0002
Prediction: [0, 2, 1]
Target:     [0, 2, 1]
Score: True

Video: animation_0003
Prediction: [2, 1, 0]
Target:     [2, 1, 0]
Score: True

Video: animation_0004
Prediction: [2, 0, 1]
Target:     [2, 0, 1]
Score: True

Video: animation_0005
Prediction: [0, 1, 2]
Target:     [0, 1, 2]
Score: True

Video: animation_0006
Prediction: [1, 2, 0]
Target:     [1, 2, 0]
Score: True

Video: animation_0007
Prediction: [0, 2, 1]
Target:     [0, 2, 1]
Score: True

Video: animation_0008
Prediction: [2, 1, 0]
Target:     [2, 1, 0]
Score: True

Video: animation_0009
Prediction: [2, 0, 1]
Target:     [2, 0, 1]
Score: True

Video: animation_0010
Prediction: [1, 0, 2]
Target:     [1, 0, 2]
Score: True

Video: animation_0011
Prediction: [1, 0, 2]
Target:     [1, 0, 2]
Score: True

Video: animation_0012
Prediction: [2, 0, 1]
Target: 

In [ ]:
# zapisz swój raport do zmiennej poniżej, abyśmy mogli go później automatycznie odczytać sprawdzaczką
raport_2 = \
"""
Raport z zadania:

Ogólna idea jest taka jak w moim rozwiązaniu pierwszej części, czyli algorytmicznie liczę na podstawie macierzy odległości
które współrzędne z poprzedniej klatki odpowiadają którym współrzędnym bierzącej klatki, jednak jest pare różnic:

- Operuję na środkach boxów ograniczających, a nie wierzchołkach
- Dodałem wektory ruchu - dla każdego kubeczka liczę wektor przemieszczenia (kierunek, zwrot i dystans) w którym się
przemieszcza gdy przypisuję mu nowego boxa ograniczającego, a gdy nie przypisuję mu, żadnego boxa z danej klatki to
przesówam współrzędne o ten wektor (tablica 'vectors' w kodzie)
- Dodałem minimalną róznicę wymaganą by przypisać boxa oparty o to jak ile klatek wcześniej przypisano ostatnio boxa do
danego kubeczka (gdy kubeczki są za blisko siebie lepiej przesunąć ich boxy z poprzedniego przypisania o wektor niż losowo
przypisać boxa do jednego z nich) (funkcja 'is_good_distance')
- Algorytm przypisywania w funkcji 'move' uległ zmianie - teraz w pierwszej kolejnościu szuka dla którego kubeczka przypisanie
jest najbardzije pewne (który box ograniczający można jednoznacznie przypisać do kubeczka)

Zachęcam do rzucenia okiem na kod chociaż zdaję sobię sprawę, że jest troszeczkę nieczytelny i dziękuję za przeczytanie mojego referatu w całości :)
"""